https://github.com/llmware-ai/llmware/blob/main/examples/invoice_processing.py


In [1]:

#   invoice_processing.py example illustrates the use of parsing combined with prompts_with_sources
#   to rapidly iterate through a batch of invoices and ask a set of questions, and then save the full
#   output to both (1) .jsonl for integration into an upstream application/database and (2) to a CSV
#   for human review in excel.

#   note: the sample code pulls from a public repo to load the sample invoice documents the first time -
#   please feel free to substitute with your own invoice documents (PDF/DOCX/PPTX/XLSX/CSV/TXT) if you prefer.

#   this example does not require a database or embedding

#   this example can be run locally on a laptop by setting 'run_on_cpu=True'
#   if 'run_on_cpu==False", then please see the example 'launch_llmware_inference_server.py'
#   to configure and set up a 'pop-up' GPU inference server in just a few minutes

In [2]:
import os
import re

from llmware.prompts import Prompt, HumanInTheLoop
from llmware.configs import LLMWareConfig
from llmware.setup import Setup
from llmware.models import ModelCatalog

In [3]:
def invoice_processing(run_on_cpu=True):

    #   Step 1 - Pull down the sample files from S3 through the .load_sample_files() command
    #   --note: if you need to refresh the sample files, set 'over_write=True'
    print("update: Downloading Sample Files")

    sample_files_path = Setup().load_sample_files(over_write=False)
    invoices_path = os.path.join(sample_files_path, "Invoices")

    #   Step 2 - simple sample query list - each question will be asked to each invoice
    query_list = ["What is the total amount of the invoice?",
                  "What is the invoice number?",
                  "What are the names of the two parties?"]

    #   Step 3 - Load Model

    if run_on_cpu:

        #   load local bling model that can run on cpu/laptop
        #   alternates to try on cpu:
        #       -- "llmware/bling-1.4b-0.1", "llmware/bling-falcon-1b-0.1",
        #       -- "llmware/bling-sheared-llama-2.7b-0.1", "llmware/bling-sheared-llama-1.3b-0.1",
        #       -- "llmware/bling-red-pajamas-3b-0.1", "llmware/bling-stable-lm-3b-4e1t-0.1"

        #   note: bling-1b-0.1 is the *fastest* & *smallest*, but will make more errors than larger BLING models
        model_name = "llmware/bling-1b-0.1"

    else:

        #   use GPU-based inference server to process
        #  *** see the launch_llmware_inference_server.py example script to setup ***

        server_uri_string = "http://11.123.456.789:8088"    # insert your server_uri_string
        server_secret_key = "demo-test"
        ModelCatalog().setup_custom_llmware_inference_server(server_uri_string, secret_key=server_secret_key)
        model_name = "llmware-inference-server"

    #   attach inference server to prompt object
    prompter = Prompt().load_model(model_name)

    #   Step 4 - main loop thru folder of invoices

    for i, invoice in enumerate(os.listdir(invoices_path)):

        #   just in case (legacy on mac os file system - not needed on linux or windows)
        if invoice != ".DS_Store":

            print("\nAnalyzing invoice: ", str(i + 1), invoice)

            for question in query_list:

                #   Step 4A - parses the invoices in memory and attaches as a source to the Prompt
                source = prompter.add_source_document(invoices_path,invoice)

                #   Step 4B - executes the prompt on the LLM (with the loaded source)
                output = prompter.prompt_with_source(question,prompt_name="default_with_context")

                for i, response in enumerate(output):
                    print("LLM Response - ", question, " - ", re.sub("[\n]"," ", response["llm_response"]))

                prompter.clear_source_materials()

    # Save jsonl report with full transaction history to /prompt_history folder
    print("\nupdate: prompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(),prompter.prompt_id))

    prompter.save_state()

    # Generate CSV report for easy Human review in Excel
    csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()

    print("\nupdate: csv output for human review - ", csv_output)

    return 0

In [4]:
invoice_processing(run_on_cpu=True)

update: Downloading Sample Files


config.json:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

C:\Users\p2p2l\anaconda3\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\p2p2l\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/4.11G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/33/13/33136e32389dad1ccba2a3abb0a18c760fcde7f5c0c4b30726ae34007dfaf416/965e95b9ec68af2e704fc1f3ddd4215081b5cc91dd122b8cdae652ce8eb629a4?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1702351829&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjM1MTgyOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMy8xMy8zMzEzNmUzMjM4OWRhZDFjY2JhMmEzYWJiMGExOGM3NjBmY2RlN2Y1YzBjNGIzMDcyNmFlMzQwMDdkZmFmNDE2Lzk2NWU5NWI5ZWM2OGFmMmU3MDRmYzFmM2RkZDQyMTUwODFiNWNjOTFkZDEyMmI4Y2RhZTY1MmNlOGViNjI5YTQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=p3zaUwSEwjUMeEunuz7s7weDy66ZoetpqtQgrnYPimCH7YFE6S61rJoEisakOX290ufu2%7E3g97mRHPPuOZdguHXjMZ3MCwHsFEAeFI70giXc-LzvM642QxLWI6F2I8JX85c62u2ud3XvVClcnCIgXLLz46jeIDR

...

pytorch_model.bin:  89%|########9 | 3.67G/4.11G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]


Analyzing invoice:  2 Alpha Services Vendor Inc Invoice.pdf
LLM Response -  What is the total amount of the invoice?  -   $22,500.00
LLM Response -  What is the invoice number?  -   0001
LLM Response -  What are the names of the two parties?  -   Alpha Inc. and Services Vendor Inc.

Analyzing invoice:  3 Alpha Simple Invoice Template.pdf
LLM Response -  What is the total amount of the invoice?  -   $12,980.00
LLM Response -  What is the invoice number?  -   No.  1.
LLM Response -  What are the names of the two parties?  -   Alpha Inc.

Analyzing invoice:  4 Beta Services Vendor Inc Invoice.pdf
LLM Response -  What is the total amount of the invoice?  -   33,500.00
LLM Response -  What is the invoice number?  -   0002
LLM Response -  What are the names of the two parties?  -   Beta Inc. and Beta Project

Analyzing invoice:  5 Beta Simple Invoice Template.pdf
LLM Response -  What is the total amount of the invoice?  -   $880.00
LLM Response -  What is the invoice number?  -   2, 2022
LL

LLM Response -  What is the total amount of the invoice?  -   98,000.00
LLM Response -  What is the invoice number?  -   0015
LLM Response -  What are the names of the two parties?  -   Omicron Inc. and Services Vendor Inc.

Analyzing invoice:  27 Phi Services Vendor Inc Invoice.pdf
LLM Response -  What is the total amount of the invoice?  -   $70,500.00
LLM Response -  What is the invoice number?  -   0017
LLM Response -  What are the names of the two parties?  -   1.  Phi Inc.   2.  Phi Project   3.  Phi Project   4.  Phi Project   5.  Phi Project   6.  Phi Project   7.  Phi Project   8.  Phi Project   9.  Phi Project   10.  Phi Project   11.  Phi Project   12.  Phi Project   13.  Phi Project   14.  Phi Project   15.  Phi Project   16.  Phi Project   17.  Phi Project   18.  Phi Project   19.  Phi Project   20.  Phi Project   21.  Phi Project   22.  Phi Project   23.  Phi Project   24.  Phi Project   25.  Phi Project   26.  Phi Project   27.  Phi Project   28.  Phi Project   29.  Phi 

OSError: [Errno 22] Invalid argument: 'C:\\Users\\p2p2l\\llmware_data\\prompt_history\\interaction_report_Fri Dec  8 23:05:09 2023.csv'